In [150]:
import pandas as pd
import numpy as np
import pylogit
from scipy.special import logit
from sklearn.metrics import brier_score_loss, mean_squared_error
import statsmodels.formula.api as smf
from collections import Counter
pd.options.display.max_columns = 100

In [91]:
def mlogit(formula, df, obs_id, alt_id) :
    """
    df must be sorted by obs_id to use this function
    """
    from patsy import dmatrices
    from collections import OrderedDict
    import pylogit as pl
    import numpy as np
    data = dmatrices(formula+' -1', df, return_type = "dataframe")
    all_alts = sorted(df.loc[:,alt_id].unique())
    
    spec = OrderedDict()
    names = OrderedDict()
    spec['intercept'] = all_alts[1:]
    names['intercept'] = [f'(Intercept):{alt}' for alt in all_alts[1:]]
    c_names = data[1].design_info.column_names
    choice = data[0].design_info.column_names[0]
    data[1][choice] = data[0]
    data[1][[obs_id,alt_id]] = df[[obs_id,alt_id]]    
    for c in c_names :
        spec[c] = [all_alts]
        names[c] = [c]
    model = pl.create_choice_model(data = data[1], alt_id_col = alt_id, obs_id_col = obs_id, 
                                   choice_col = choice, specification = spec, names = names, model_type = 'MNL')
    model.fit_mle(np.zeros(model.design.shape[1]), method = 'Powell')
    return model

In [92]:
df = pd.read_csv('xc.csv', parse_dates = ['dt'])
df = df.rename(columns = {'dt' : 'date'})
df = df.loc[df.race_type == 'flat_race']
df = df.sort_values('date')

In [93]:
# dups = df.groupby('race_id').box.agg(lambda x : len(x)-x.nunique())
# dups = dups.loc[dups>0]
# df = df.loc[~df.race_id.isin(dups.index)]
win_choice = df.loc[df.winner == 1].sample(frac = 1.0).groupby('race_id').head(1).loc[:,['race_id','box']].copy()
win_choice['twinner'] = 1
df = df.merge(win_choice, on = ['race_id','box'], how = 'left')
df.twinner = df.twinner.fillna(0)
df = df.sort_values(['date','box'])

In [94]:
startup=1.75
distdefault=400
distexponent=0.11

df['race_num'] = df.groupby('dog_id').cumcount()
df['min_race_num'] = df.groupby('race_id').race_num.transform('min')
df['rest'] = df.groupby('dog_id').date.diff()
df['max_rest'] = df.groupby('race_id').rest.transform('max')
df['max_box'] = df.groupby('race_id').box.transform('max')
df['runtime']= df.time - startup

df['mps'] = df.distance_m/df.time
df['avg_mps'] = df.groupby('dog_id').mps.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))
df['mmps'] = df.distance_m/df.runtime * (df.distance_m**distexponent)/(distdefault**distexponent)
df['avg_mmps'] = df.groupby('dog_id').mmps.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))



In [237]:
df['avg_going'] = df.groupby('dog_id').going.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))
df['avg_dec_price'] = df.groupby('dog_id').decimal_price.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))

In [95]:
# df['dt'] = df['dt'].astype('datetime64[ns]')
# df['prev_dt'] = df['dt'].astype('datetime64[ns]')
# df['prev_dt'] = df.groupby('dog_id').dt.transform(lambda x : x.shift(1, fill_value = 0))
# df['last_race'] = (df.dt-df.prev_dt).dt.days
# df['prev_races'] = df.groupby('dog_id').cumcount()

In [96]:
alpha = 0.03
halflife = pd.Timedelta(-np.log(2)/np.log(1-alpha), unit = 'Days')
df['ema_mmps'] = df.groupby('dog_id').apply(
    lambda x : x.mmps.ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)

df['ema_mps'] = df.groupby('dog_id').apply(
    lambda x : x.mps.ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)

In [97]:
test = pd.get_dummies(data=df.stadium_id)
test2=df.copy(deep=True)
df = pd.merge(test2, test, on = test2.index, how = 'left')

In [238]:
df.head()

,key_0,Unnamed: 0,race_id,stadium_id,distance_m,going,race_type,race_grade,dog_id,place,time,decimal_price,comment,box,kg,winner,date,origTime,dnf,twinner,race_num,min_race_num,rest,max_rest,max_box,runtime,mps,avg_mps,mmps,avg_mmps,ema_mmps,ema_mps,13003,13004,13007,13008,13009,13010,13013,13014,13019,13020,13021,13023,13025,13026,13035,13037,13043,13048,13053,13059,13061,i13009,i13037,i13003,i13004,i13007,i13008,i13010,i13013,i13014,i13019,i13020,i13021,i13023,i13025,i13026,i13035,i13043,i13048,i13053,i13059,i13061,Crowded,EP,Wide,Active,Bump,iCrowded,iEP,iWide,iBump,iActive,avg_going,avg_dec_price
0,7,371,4418779,13025,515,-0.1,flat_race,A7,2247352,6,31.92,6.00,"Crowded1,Ck2,FinLame",1,28.00,0,2019-01-01,31.92,0,0.0,0,0,NaT,NaT,6,30.17,16.134085,0.0,17.551091,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21,357,4418777,13037,480,-0.3,flat_race,A2,2278381,4,29.63,6.00,Crowded1&3,1,31.25,0,2019-01-01,29.63,0,0.0,0,0,NaT,NaT,6,27.88,16.199798,0.0,17.565415,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,27,351,4418776,13037,264,-0.1,flat_race,D4,2172228,6,16.40,5.50,Crowded1&2,1,25.00,0,2019-01-01,16.40,0,0.0,0,0,NaT,NaT,6,14.65,16.097561,0.0,17.215361,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,29,363,4418778,13037,480,-0.3,flat_race,A3,2262271,2,29.09,7.00,"Rails,EveryChance",1,27.25,0,2019-01-01,29.09,0,0.0,0,0,NaT,NaT,6,27.34,16.500516,0.0,17.912354,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31,380,4418780,13025,515,0.0,flat_race,A3,2335182,6,30.84,3.25,Crowded&Ck3,1,27.25,0,2019-01-01,30.84,0,0.0,0,0,NaT,NaT,6,29.09,16.699092,0.0,18.202695,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
# df.columns

In [132]:
# cols = [13003,13004,13007,13008,13009,13010,13013,13014,13019,13020,13021,13023,13025,13026,13035,13037,13043,13048,13053,13059,13061]
# cols.sort()
# cols

In [133]:
df['i13003'] = df[13003] - df.groupby('dog_id').apply(
    lambda x : x[13003].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13004'] = df[13004] - df.groupby('dog_id').apply(
    lambda x : x[13004].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13007'] = df[13007] - df.groupby('dog_id').apply(
    lambda x : x[13007].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13008'] = df[13008] - df.groupby('dog_id').apply(
    lambda x : x[13008].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13009'] = df[13009] - df.groupby('dog_id').apply(
    lambda x : x[13009].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13010'] = df[13010] - df.groupby('dog_id').apply(
    lambda x : x[13010].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13013'] = df[13013] - df.groupby('dog_id').apply(
    lambda x : x[13013].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13014'] = df[13014] - df.groupby('dog_id').apply(
    lambda x : x[13014].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13019'] = df[13019] - df.groupby('dog_id').apply(
    lambda x : x[13019].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13020'] = df[13020] - df.groupby('dog_id').apply(
    lambda x : x[13020].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13021'] = df[13021] - df.groupby('dog_id').apply(
    lambda x : x[13021].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13023'] = df[13023] - df.groupby('dog_id').apply(
    lambda x : x[13023].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13025'] = df[13025] - df.groupby('dog_id').apply(
    lambda x : x[13025].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13026'] = df[13026] - df.groupby('dog_id').apply(
    lambda x : x[13026].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13035'] = df[13035] - df.groupby('dog_id').apply(
    lambda x : x[13035].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13037'] = df[13037] - df.groupby('dog_id').apply(
    lambda x : x[13037].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13043'] = df[13043] - df.groupby('dog_id').apply(
    lambda x : x[13043].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13048'] = df[13048] - df.groupby('dog_id').apply(
    lambda x : x[13048].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13053'] = df[13053] - df.groupby('dog_id').apply(
    lambda x : x[13053].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13059'] = df[13059] - df.groupby('dog_id').apply(
    lambda x : x[13059].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['i13061'] = df[13061] - df.groupby('dog_id').apply(
    lambda x : x[13061].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)

In [281]:
df['comment'] = df.comment.fillna('')
df['Crowded'] = 1*(df.comment.str.contains('Crd') | df.comment.str.contains('Crowded') | df.comment.str.contains('CrD'))
df['EP'] = 1*(df.comment.str.contains('EP|EarlyPace|EPace|Ep|EarlyP'))
df['Wide'] = 1*(df.comment.str.contains('W|Wide'))
df['Bump'] = 1*(df.comment.str.contains('Bmp|Bumped'))
df['Slow'] = 1*(df.comment.str.contains('SAw|Slow'))
df['Led'] = 1*(df.comment.str.contains('Led|Ld'))
df['Rails'] = 1*(df.comment.str.contains('Rls|Rails'))
df['Active'] = df.comment.str.count(',') + df.comment.str.count('&') + df.comment.str.count('/') + df.comment.str.count('amp')

In [289]:
df['iCrowded'] = df.groupby('dog_id').apply(
    lambda x : x['Crowded'].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['iEP'] = df.groupby('dog_id').apply(
    lambda x : x['EP'].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['iWide'] = df.groupby('dog_id').apply(
    lambda x : x['Wide'].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['iBump'] = df.groupby('dog_id').apply(
    lambda x : x['Bump'].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['iActive'] = df.groupby('dog_id').apply(
    lambda x : x['Active'].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['iSlow'] = df.groupby('dog_id').apply(
    lambda x : x['Slow'].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['iLed'] = df.groupby('dog_id').apply(
    lambda x : x['Led'].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)
df['iRails'] = df.groupby('dog_id').apply(
    lambda x : x['Rails'].ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)

In [290]:
# df['ewm'] = df.groupby('dog_id')[df.stadium_id].apply(
#     lambda x : x.ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
# ).reset_index(level = 'dog_id',drop = True)

In [291]:
bad_races = df.loc[df.groupby('race_id').box.transform(lambda x : x.nunique() < len(x))].race_id.unique()
mrn = df.min_race_num >= 3
mr = df.max_rest <= pd.Timedelta(90, unit = 'Days')
mb = df.max_box >= 4
df_fil = df.loc[mb & mrn & mr & ~df.race_id.isin(bad_races)].sort_values(['date','race_id'])

In [207]:
df_fil.head()

,key_0,Unnamed: 0,race_id,stadium_id,distance_m,going,race_type,race_grade,dog_id,place,time,decimal_price,comment,box,kg,winner,date,origTime,dnf,twinner,race_num,min_race_num,rest,max_rest,max_box,runtime,mps,avg_mps,mmps,avg_mmps,ema_mmps,ema_mps,13003,13004,13007,13008,13009,13010,13013,13014,13019,13020,13021,13023,13025,13026,13035,13037,13043,13048,13053,13059,13061,i13009,i13037,i13003,i13004,i13007,i13008,i13010,i13013,i13014,i13019,i13020,i13021,i13023,i13025,i13026,i13035,i13043,i13048,i13053,i13059,i13061,Crowded,EP,Wide,Active,Bump,iCrowded,iEP,iWide,iBump,iActive
19371,19900,20579,4428694,13026,265,-0.25,flat_race,D2,2295234,3,17.23,4.0,EvCh,1,31.25,0,2019-01-22,17.23,0,0.0,4,3,6 days,8 days,6,15.48,15.380151,15.376832,16.360831,16.357352,16.367674,15.385555,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.202994,0.000000,0.000000,0.0,0.202994
19500,19807,20578,4428694,13026,265,-0.25,flat_race,D2,2291147,6,17.63,6.0,StbStt,2,24.00,0,2019-01-22,17.63,0,0.0,4,3,7 days,8 days,6,15.88,15.031197,15.309564,15.948720,16.277365,16.264017,15.298283,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.458703,0.237836,0.000000,0.0,1.148219
19657,19884,20577,4428694,13026,265,-0.25,flat_race,D2,2264827,1,16.85,3.5,"QAw,ALd",3,33.50,1,2019-01-22,16.85,0,1.0,3,3,7 days,8 days,6,15.10,15.727003,15.368591,16.772561,16.347190,16.361066,15.380308,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.589589,0.410411,0.000000,0.0,1.000000
19803,19882,20575,4428694,13026,265,-0.25,flat_race,D2,2252689,2,17.05,4.0,"EP,EvCh",4,31.75,0,2019-01-22,17.05,0,0.0,3,3,8 days,8 days,6,15.30,15.542522,15.415221,16.553312,16.402670,16.418625,15.428669,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,0.395755,0.604245,0.000000,0.0,1.725408
19947,19883,20576,4428694,13026,265,-0.25,flat_race,D2,2260710,4,17.25,11.0,"EP,Wide",5,24.50,0,2019-01-22,17.25,0,0.0,4,3,6 days,8 days,6,15.50,15.362319,15.490651,16.339720,16.491822,16.490814,15.489803,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,1,0,0.000000,0.000000,0.457673,0.0,0.947863


In [208]:
# df_fil.loc[df_fil.dog_id == 2279880]

## Question 1

In [209]:
dftrain = df_fil.loc[(df_fil.date>'2019-06-30')&(df_fil.date<'2020-02-01')].sort_values(['race_id','box']).copy()
dftest = df_fil.loc[(df_fil.date>'2020-01-31')].sort_values(['race_id','box']).copy()

### 1a

In [210]:
model = mlogit('twinner ~ avg_mmps', dftrain, 'race_id', 'box')
model.get_statsmodels_summary()

Log-likelihood at zero: -37,996.5034
Initial Log-likelihood: -37,996.5034


/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:519: RuntimeWarning: Method Powell does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:524: RuntimeWarning: Method Powell does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/pylogit/estimation.py:678: OptimizeWarning: Unknown solver options: gtol
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 3.02 seconds.
Final log-likelihood: -37,601.2255


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               21,472
Model:             Multinomial Logit Model   Df Residuals:                   21,466
Method:                                MLE   Df Model:                            6
Date:                     Sun, 09 May 2021   Pseudo R-squ.:                   0.010
Time:                             10:15:36   Pseudo R-bar-squ.:               0.010
AIC:                            75,214.451   Log-Likelihood:            -37,601.226
BIC:                            75,262.298   LL-Null:                   -37,996.503
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
(Intercept):2    -0.0384      0.024     -1.625      0.104      -0.085       0.008
(Intercept):3     0.0186      0.023      0.796      0.426      -0.027       0.064
(Intercept):4    -0.0131      0.024     -0.557      0.578      -0.059       0.033
(Intercept):5    -0.1091      0.024     -4.515      0.000      -0.156      -0.062
(Intercept):6     0.0186      0.023      0.793      0.428      -0.027       0.065
avg_mmps          1.2413      0.046     27.036      0.000       1.151       1.331
=================================================================================
"""

### 1B

In [211]:
brier_score_loss(dftest.twinner, model.predict(dftest))

0.14006026063628393

## Question 2

In [212]:
## Say x percent of win comes from market prob and then 1-x comes from actual result

In [213]:
dftrain2 = df_fil.loc[df_fil.date.between('2019-07-01','2019-11-30')].copy()
dftest2 = df_fil.loc[df_fil.date.between('2019-12-01','2020-01-31')].copy()

### 2A

In [214]:
model2 = smf.ols('mmps ~ ema_mmps',dftrain2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mmps   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.757
Method:                 Least Squares   F-statistic:                 2.770e+05
Date:                Sun, 09 May 2021   Prob (F-statistic):               0.00
Time:                        10:15:36   Log-Likelihood:                -8333.5
No. Observations:               88779   AIC:                         1.667e+04
Df Residuals:                   88777   BIC:                         1.669e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7502      0.032     23.763      0.000       0.688       0.812
ema_mmps       0.9567      0.002    526.307      0.000       0.953       0.960
==============================================================================
Omnibus:                     8704.514   Durbin-Watson:                   1.369
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27927.859
Skew:                          -0.505   Prob(JB):                         0.00
Kurtosis:                       5.555   Cond. No.                         617.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [215]:
mean_squared_error(dftest2.mmps, model2.predict(dftest2))

0.07110777430247221

### 2B

In [216]:
# df_fil.columns

In [217]:
model2b = smf.ols('mmps ~ ema_mmps+i13009+i13037+i13003+i13004+i13007+i13008+i13010+i13013+i13014+i13019+i13020+i13021+i13023+i13025+i13026+i13035+i13043+i13048+i13053+i13059+i13061',dftrain2).fit()
model2b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mmps   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                 1.664e+04
Date:                Sun, 09 May 2021   Prob (F-statistic):               0.00
Time:                        10:15:36   Log-Likelihood:                -312.47
No. Observations:               88779   AIC:                             668.9
Df Residuals:                   88757   BIC:                             875.6
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6412      0.029     22.205      0.000       0.585       0.698
ema_mmps       0.9630      0.002    579.118      0.000       0.960       0.966
i13009        -0.2210      0.010    -21.447      0.000      -0.241      -0.201
i13037         0.3806      0.012     32.266      0.000       0.357       0.404
i13003        -0.0806      0.018     -4.601      0.000      -0.115      -0.046
i13004         0.1434      0.012     12.025      0.000       0.120       0.167
i13007        -0.5092      0.012    -41.047      0.000      -0.534      -0.485
i13008        -0.0825      0.022     -3.835      0.000      -0.125      -0.040
i13010         0.6529      0.012     55.076      0.000       0.630       0.676
i13013         0.1916      0.029      6.705      0.000       0.136       0.248
i13014         0.1524      0.018      8.520      0.000       0.117       0.187
i13019        -0.0800      0.012     -6.866      0.000      -0.103      -0.057
i13020        -0.1430      0.019     -7.692      0.000      -0.179      -0.107
i13021        -0.0286      0.050     -0.572      0.568      -0.127       0.070
i13023         0.1208      0.017      6.989      0.000       0.087       0.155
i13025         0.5370      0.012     44.627      0.000       0.513       0.561
i13026        -0.3610      0.011    -32.615      0.000      -0.383      -0.339
i13035         0.3365      0.020     16.512      0.000       0.297       0.376
i13043         0.0780      0.014      5.414      0.000       0.050       0.106
i13048        -0.8009      0.013    -62.940      0.000      -0.826      -0.776
i13053         0.1296      0.040      3.235      0.001       0.051       0.208
i13059         0.1752      0.031      5.694      0.000       0.115       0.236
i13061        -0.5909      0.013    -44.386      0.000      -0.617      -0.565
==============================================================================
Omnibus:                     7728.238   Durbin-Watson:                   1.380
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16239.130
Skew:                          -0.571   Prob(JB):                         0.00
Kurtosis:                       4.757   Cond. No.                     4.46e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.35e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [218]:
mean_squared_error(dftest2.mmps, model2b.predict(dftest2))

0.0648785166511093

## Question 3

In [219]:
## Look at number of commas in comments to see how "active" they were in the race

In [220]:
# df_fil['comment'] = df_fil.comment.fillna('')

In [221]:
# df_fil['Crowded'] = 1*(df_fil.comment.str.contains('Crd') | df_fil.comment.str.contains('Crowded') | df_fil.comment.str.contains('CrD'))
# df_fil['EP'] = 1*(df_fil.comment.str.contains('EP|EarlyPace|EPace|Ep|EarlyP'))
# df_fil['Wide'] = 1*(df_fil.comment.str.contains('W|Wide'))
# df_fil['Bump'] = 1*(df_fil.comment.str.contains('Bmp|Bumped'))
# df_fil['Active'] = df_fil.comment.str.count(',') + df_fil.comment.str.count('&') + df_fil.comment.str.count('/') + df_fil.comment.str.count('amp')

In [292]:
dftrain3 = df_fil.loc[df_fil.date.between('2019-07-01','2019-11-30')].copy()
dftest3 = df_fil.loc[df_fil.date.between('2019-12-01','2020-01-31')].copy()

In [293]:
model3 = smf.ols('mmps ~ ema_mmps+i13009+i13037+i13003+i13004+i13007+i13008+i13010+i13013+i13014+i13019+i13020+i13021+i13023+i13025+i13026+i13035+i13043+i13048+i13053+i13059+i13061+iCrowded+iEP+iActive+iBump+iLed+iSlow+iRails',dftrain3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mmps   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                 1.265e+04
Date:                Mon, 10 May 2021   Prob (F-statistic):               0.00
Time:                        07:38:56   Log-Likelihood:                 166.54
No. Observations:               88779   AIC:                            -275.1
Df Residuals:                   88750   BIC:                            -2.658
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4330      0.031     13.908      0.000       0.372       0.494
ema_mmps       0.9738      0.002    549.050      0.000       0.970       0.977
i13009        -0.2179      0.010    -21.249      0.000      -0.238      -0.198
i13037         0.3837      0.012     32.705      0.000       0.361       0.407
i13003        -0.0841      0.017     -4.826      0.000      -0.118      -0.050
i13004         0.1471      0.012     12.398      0.000       0.124       0.170
i13007        -0.5046      0.012    -40.885      0.000      -0.529      -0.480
i13008        -0.0799      0.021     -3.733      0.000      -0.122      -0.038
i13010         0.6524      0.012     55.319      0.000       0.629       0.676
i13013         0.1888      0.028      6.643      0.000       0.133       0.245
i13014         0.1487      0.018      8.362      0.000       0.114       0.184
i13019        -0.0825      0.012     -7.118      0.000      -0.105      -0.060
i13020        -0.1430      0.018     -7.733      0.000      -0.179      -0.107
i13021        -0.0351      0.050     -0.704      0.481      -0.133       0.063
i13023         0.1198      0.017      6.971      0.000       0.086       0.154
i13025         0.5412      0.012     45.209      0.000       0.518       0.565
i13026        -0.3604      0.011    -32.734      0.000      -0.382      -0.339
i13035         0.3358      0.020     16.567      0.000       0.296       0.376
i13043         0.0796      0.014      5.558      0.000       0.052       0.108
i13048        -0.7995      0.013    -63.155      0.000      -0.824      -0.775
i13053         0.1251      0.040      3.140      0.002       0.047       0.203
i13059         0.1742      0.031      5.690      0.000       0.114       0.234
i13061        -0.5895      0.013    -44.512      0.000      -0.615      -0.564
iCrowded       0.0567      0.004     14.647      0.000       0.049       0.064
iEP           -0.0145      0.004     -3.327      0.001      -0.023      -0.006
iActive        0.0079      0.002      5.143      0.000       0.005       0.011
iBump          0.0644      0.005     13.095      0.000       0.055       0.074
iLed          -0.0525      0.004    -12.321      0.000      -0.061      -0.044
iSlow          0.0154      0.004      3.486      0.000       0.007       0.024
iRails        -0.0113      0.003     -4.177      0.000      -0.017      -0.006
==============================================================================
Omnibus:                     7484.880   Durbin-Watson:                   1.382
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15567.648
Skew:                          -0.558   Prob(JB):                         0.00
Kurtosis:                       4.721   Cond. No.                     1.00e+16
==============================================================================

N

In [294]:
mean_squared_error(dftest3.mmps, model3.predict(dftest3))

0.06456782565092986

## Question 4

### 4.A

In [295]:
df_fil['mmps_forecast'] = model3.predict(df_fil)

In [296]:
dftrain4a = df_fil.loc[df_fil.date<'2020-02-01'].sort_values(['race_id','box']).copy()
dftest4a = df_fil.loc[(df_fil.date>'2020-01-31')].sort_values(['race_id','box']).copy()

In [297]:
model4a = mlogit('twinner ~ mmps_forecast', dftrain4a, 'race_id', 'box')
model4a.get_statsmodels_summary()

Log-likelihood at zero: -59,637.0868
Initial Log-likelihood: -59,637.0868


/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:519: RuntimeWarning: Method Powell does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:524: RuntimeWarning: Method Powell does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/pylogit/estimation.py:678: OptimizeWarning: Unknown solver options: gtol
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 4.69 seconds.
Final log-likelihood: -58,234.3361


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               33,649
Model:             Multinomial Logit Model   Df Residuals:                   33,643
Method:                                MLE   Df Model:                            6
Date:                     Mon, 10 May 2021   Pseudo R-squ.:                   0.024
Time:                             07:43:10   Pseudo R-bar-squ.:               0.023
AIC:                           116,480.672   Log-Likelihood:            -58,234.336
BIC:                           116,531.215   LL-Null:                   -59,637.087
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
(Intercept):2    -0.0447      0.019     -2.355      0.019      -0.082      -0.008
(Intercept):3     0.0410      0.019      2.191      0.028       0.004       0.078
(Intercept):4    -0.0226      0.019     -1.192      0.233      -0.060       0.015
(Intercept):5    -0.1307      0.019     -6.716      0.000      -0.169      -0.093
(Intercept):6     0.0118      0.019      0.623      0.533      -0.025       0.049
mmps_forecast     3.1616      0.063     50.507      0.000       3.039       3.284
=================================================================================
"""

In [298]:
brier_score_loss(dftest4a.twinner, model4a.predict(dftest4a))

0.13847283237737143

### 4.B

In [299]:
dftrain4a.head()

,key_0,Unnamed: 0,race_id,stadium_id,distance_m,going,race_type,race_grade,dog_id,place,time,decimal_price,comment,box,kg,winner,date,origTime,dnf,twinner,race_num,min_race_num,rest,max_rest,max_box,runtime,mps,avg_mps,mmps,avg_mmps,ema_mmps,ema_mps,13003,13004,13007,13008,13009,13010,13013,13014,13019,13020,13021,13023,13025,13026,13035,13037,13043,13048,13053,13059,13061,i13009,i13037,i13003,i13004,i13007,i13008,i13010,i13013,i13014,i13019,i13020,i13021,i13023,i13025,i13026,i13035,i13043,i13048,i13053,i13059,i13061,Crowded,EP,Wide,Active,Bump,iCrowded,iEP,iWide,iBump,iActive,avg_going,avg_dec_price,Slow,Led,Rails,iSlow,iLed,iRails,mmps_forecast
19371,19900,20579,4428694,13026,265,-0.25,flat_race,D2,2295234,3,17.23,4.0,EvCh,1,31.25,0,2019-01-22,17.23,0,0.0,4,3,6 days,8 days,6,15.48,15.380151,15.376832,16.360831,16.357352,16.367674,15.385555,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.202994,0.000000,0.000000,0.0,0.202994,-0.137500,4.437500,0,0,0,0.301613,0.000000,0.0,16.388881
19500,19807,20578,4428694,13026,265,-0.25,flat_race,D2,2291147,6,17.63,6.0,StbStt,2,24.00,0,2019-01-22,17.63,0,0.0,4,3,7 days,8 days,6,15.88,15.031197,15.309564,15.948720,16.277365,16.264017,15.298283,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.458703,0.237836,0.000000,0.0,1.148219,-0.100000,4.250000,0,0,0,0.000000,0.303461,0.0,16.285862
19657,19884,20577,4428694,13026,265,-0.25,flat_race,D2,2264827,1,16.85,3.5,"QAw,ALd",3,33.50,1,2019-01-22,16.85,0,1.0,3,3,7 days,8 days,6,15.10,15.727003,15.368591,16.772561,16.347190,16.361066,15.380308,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.589589,0.410411,0.000000,0.0,1.000000,-0.083333,5.666667,0,1,0,0.000000,0.000000,0.0,16.400051
19803,19882,20575,4428694,13026,265,-0.25,flat_race,D2,2252689,2,17.05,4.0,"EP,EvCh",4,31.75,0,2019-01-22,17.05,0,0.0,3,3,8 days,8 days,6,15.30,15.542522,15.415221,16.553312,16.402670,16.418625,15.428669,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,0.395755,0.604245,0.000000,0.0,1.725408,-0.166667,6.000000,0,0,0,0.000000,0.395755,0.0,16.427239
19947,19883,20576,4428694,13026,265,-0.25,flat_race,D2,2260710,4,17.25,11.0,"EP,Wide",5,24.50,0,2019-01-22,17.25,0,0.0,4,3,6 days,8 days,6,15.50,15.362319,15.490651,16.339720,16.491822,16.490814,15.489803,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,1,0,0.000000,0.000000,0.457673,0.0,0.947863,-0.075000,9.250000,0,0,0,0.000000,0.490190,0.0,16.472764


In [318]:
model4b = mlogit('twinner ~ mmps_forecast + avg_dec_price + ema_mmps + avg_mps', dftrain4a, 'race_id', 'box')
model4b.get_statsmodels_summary()

Log-likelihood at zero: -59,637.0868
Initial Log-likelihood: -59,637.0868


/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:519: RuntimeWarning: Method Powell does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:524: RuntimeWarning: Method Powell does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/pylogit/estimation.py:678: OptimizeWarning: Unknown solver options: gtol
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 10.72 seconds.
Final log-likelihood: -57,864.6613


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               33,649
Model:             Multinomial Logit Model   Df Residuals:                   33,640
Method:                                MLE   Df Model:                            9
Date:                     Mon, 10 May 2021   Pseudo R-squ.:                   0.030
Time:                             07:52:48   Pseudo R-bar-squ.:               0.030
AIC:                           115,747.323   Log-Likelihood:            -57,864.661
BIC:                           115,823.136   LL-Null:                   -59,637.087
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
(Intercept):2    -0.0462      0.019     -2.428      0.015      -0.084      -0.009
(Intercept):3     0.0387      0.019      2.059      0.039       0.002       0.075
(Intercept):4    -0.0175      0.019     -0.922      0.356      -0.055       0.020
(Intercept):5    -0.1116      0.020     -5.719      0.000      -0.150      -0.073
(Intercept):6     0.0275      0.019      1.450      0.147      -0.010       0.065
mmps_forecast     2.9193      0.094     31.051      0.000       2.735       3.104
avg_dec_price    -0.1056      0.005    -22.659      0.000      -0.115      -0.096
ema_mmps         -0.3211      0.085     -3.786      0.000      -0.487      -0.155
avg_mps           0.7117      0.064     11.206      0.000       0.587       0.836
=================================================================================
"""

In [319]:
brier_score_loss(dftest4a.twinner, model4b.predict(dftest4a))

0.13778315206617242

### 4C

In [320]:
modelmarket = mlogit('twinner ~ decimal_price', dftrain4a, 'race_id', 'box')

Log-likelihood at zero: -59,634.6019
Initial Log-likelihood: -59,634.6019


/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:519: RuntimeWarning: Method Powell does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:524: RuntimeWarning: Method Powell does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/pylogit/estimation.py:678: OptimizeWarning: Unknown solver options: gtol
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 5.63 seconds.
Final log-likelihood: -55,049.0270


In [321]:
df_fil['final_model'] = model4b.predict(df_fil)
df_fil['market_model'] = modelmarket.predict(df_fil)

In [322]:
dftrain4c = df_fil.loc[(df_fil.date>'2019-06-30')&(df_fil.date<'2020-02-01')].sort_values(['race_id','box']).copy()
dftest4c = df_fil.loc[(df_fil.date>'2020-01-31')].sort_values(['race_id','box']).copy()

In [323]:
model4c = mlogit('twinner ~ final_model + market_model', dftrain4c, 'race_id', 'box')

Log-likelihood at zero: -37,996.5034
Initial Log-likelihood: -37,996.5034


/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:519: RuntimeWarning: Method Powell does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:524: RuntimeWarning: Method Powell does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
/Users/saumya/miniconda3/lib/python3.8/site-packages/pylogit/estimation.py:678: OptimizeWarning: Unknown solver options: gtol
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 5.36 seconds.
Final log-likelihood: -34,856.0960


In [324]:
model4c.get_statsmodels_summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               21,472
Model:             Multinomial Logit Model   Df Residuals:                   21,465
Method:                                MLE   Df Model:                            7
Date:                     Mon, 10 May 2021   Pseudo R-squ.:                   0.083
Time:                             07:55:08   Pseudo R-bar-squ.:               0.082
AIC:                            69,726.192   Log-Likelihood:            -34,856.096
BIC:                            69,782.013   LL-Null:                   -37,996.503
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
(Intercept):2     0.0065      0.024      0.265      0.791      -0.041       0.054
(Intercept):3    -0.0229      0.024     -0.946      0.344      -0.070       0.025
(Intercept):4 -3.029e-05      0.024     -0.001      0.999      -0.048       0.048
(Intercept):5     0.0221      0.025      0.884      0.377      -0.027       0.071
(Intercept):6    -0.0030      0.024     -0.123      0.902      -0.051       0.045
final_model       0.6344      0.145      4.379      0.000       0.350       0.918
market_model      6.0093      0.098     61.164      0.000       5.817       6.202
=================================================================================
"""

In [325]:
brier_score_loss(dftest4c.twinner, model4c.predict(dftest4c))

0.1317789174665306